## 9.2 长短期记忆网络(LSTM)
长期以来，隐变量模型存在长期信息保存和短期输入缺失的问题。解决这一问题的最早方法之一是长短期存储器(long short-term memory, LSTM)。它有许多与门控循环单元一样的属性。有趣的是，长短期记忆网络的设计比门控循环单元稍微复杂一些，却比门控循环单元早诞生了20年。

### 9.2.1 门控记忆元
可以说，长短期记忆网络的设计灵感来自计算机的逻辑门。长短期记忆网络引入了记忆元(memory cell)，或简称为单元(cell)。有些文献认为记忆元是隐状态的一种特殊类型，它们与隐状态具有相同的形状，其设计目的是用于记录附加的信息。为了控制记忆元，我们需要许多门。其中一个门用来从单元中输出条目，我们将其称为输出门(output gate)。另外一个门用来决定何时将数据读入单元，我们将其称为输入门(input gate)。我们还需要一种机制来重置单元的内容，由遗忘门(forget gate)来管理，这种设计的动机与门控循环单元相同，能够通过专用机制决定什么时候记忆或忽略隐状态中的输入。让我们看看这在实践中是如何运作的。


**输入门、忘记门和输出门**

就如在门控循环单元中一样，当时间步的输入和前一个时间步的隐状态作为数据送入长短期记忆网络的门中，如图9.2.1所示。它们由三个具有sigmoid激活函数的全连接层处理，以计算输入门、遗忘门和输出门的值。因此，这三个门的值都在(0, 1)的范围内。


<div align=center>
<img src='../../pics/9_2_1.jpeg' width='50%'>
</div>

我们来细化一下长短期记忆网络的数学表达。假设有h个隐藏单元，批量大小为n，输入数为d。因此，输入为$X_t \in \mathbb R^{n \times d}$，前一时间步的隐状态为$H_{t-1} \in \mathbb R ^{n \times h}$。相应地，时间步t的门被定义如下：输入门是$I_t \in \mathbb R^{n \times h}$，遗忘门是$F_t \in \mathbb R^{n \times h}$，输出门是$O_t \in \mathbb R^{n \times h}$。它们的计算方法如下：
$$
I_t = \sigma(X_t W_{xi} + H_{t-1} W_{hi}+ b_i),\\
F_t = \sigma(X_t W_{xf} + H_{t-1} W_{hf}+ b_f),\\
O_t = \sigma(X_t W_{xo} + H_{t-1} W_{ho}+ b_o),
\tag{9.2.1}
$$

其中$W_{xi}, W_{xf}, W_{xo} \in \mathbb R^{d \times h}$和$W_{hi}, W_{hf}, W_{ho} \in \mathbb R^{h \times h}$是权重参数，$b_i, b_f, b_o \in \mathbb R^{1 \times h}$是偏置参数。


**候选记忆元**

由于还没有指定各种门的操作，所以先介绍候选记忆元(candidate memory cell)$\tilde C_t \in \mathbb R^{n \times h}$。它的计算与上面描述的三个门的计算类似，但是使用tanh函数作为激活函数，函数的值范围为(-1, 1)。下面导出在时间步t处的方程：
$$
\tilde C_t = tanh(X_t W_{xc} + H_{t-1} W_{hc} + b_c),
\tag{9.2.2}
$$

其中$W_{xc} \in \mathbb R^{d \times h}$和$W_{hc} \in \mathbb R^{h \times h}$是权重参数，$b_c \in \mathbb R^{1 \times h}$是偏置参数。

候选记忆元的如图9.2.2所示
<div align=center>
<img src='../../9_2_2.jpeg' width='50%'>
</div>

**记忆元**

在门控循环单元中，有一种机制来控制输入和遗忘(或跳过)。类似地，在长短期记忆网络中，也有两个门用于这样的目的：输入门$I_t$控制采用多少来自$\tilde C_t$的新数据，而遗忘门$F_t$控制保留多少过去的记忆元$C_{t-1} \in \mathbb R^{n \times h}$的内容。使用按元素乘法，得出：
$$
C_t = F_t \odot C_{t-1} + I_t \odot \tilde C_t 
\tag{9.2.3}
$$